In [1]:
!pip install SimpleITK

In [2]:
import keras
import tensorflow as tf
import json, os, time, copy
import numpy as np
import skimage
import seaborn as sns
import glob
import SimpleITK as sitk
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

sns.set()


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from random import randint
import cv2


class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    #Have to set parameters here
    def __init__(self, list_IDs, labels, batch_size=32, dim=(128,128,128), n_channels=3,
                 n_classes=7, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
 
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, 128, 128 , self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        
        

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            #X[i,] = np.load('data/' + ID + '.npy')
            image = sitk.ReadImage(ID) 
        
           # im = sitk.GetImageFromArray(image)

            num_dim = image.GetDimension()

            orig_pixelid = image.GetPixelIDValue()
            orig_origin = image.GetOrigin()
            orig_direction = image.GetDirection()
            orig_spacing = np.array(image.GetSpacing())
            orig_size = np.array(image.GetSize(), dtype=np.int)



            new_size = [128,128,128] #  SimpleITK expects lists, not ndarrays

            new_spacing = [sz*spc/nsz for nsz,sz,spc in zip(new_size, image.GetSize(), image.GetSpacing())]

            resample_filter = sitk.ResampleImageFilter()

            resampled_sitk_image = resample_filter.Execute(image,
                                                           new_size,
                                                           sitk.Transform(),
                                                           sitk.sitkNearestNeighbor,
                                                           orig_origin,
                                                           new_spacing,
                                                           orig_direction,
                                                           fill_value,
                                                           orig_pixelid)

           # resampled_sitk_image = np.expand_dims (resampled_sitk_image, axis =2)



            imageArray = sitk.GetArrayFromImage(resampled_sitk_image)

            angle = 0.0
            scale = 0
            rangle = np.random.random() * angle
            sc = (np.random.random() - 0.5) * 2 * scale + 1.0
            rows, cols = (128,128)
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), rangle, sc)
            
            warped_img = np.expand_dims(cv2.warpAffine(imageArray[32+randint(0,64)], M, (cols, rows), flags=0), -1)
            #X[i]= cv2.merge(warped_img, warped_img, warped_img)
            #print(X[i].shape)

            for j in range (128):
              for k in range (128):
                for l in range(2):
                  X[i][j][k][l] = warped_img[j][k][0]


            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from keras.models import Sequential


# Parameters - Have to fix this
params = {'dim': (128,128,128),
          'batch_size': 32,
          'n_classes': 7,
          'n_channels': 3,
          'shuffle': True}

labels = {}
partition = {}

rootdir = "/content/drive/My Drive/Classifier_train/"
train_niiPaths = []
num_files= len(train_niiPaths)


for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file.endswith("nii.gz")):
                train_niiPaths.append(os.path.join(subdir, file))

print("Number of source filePaths: " + str(len(train_niiPaths)))


for fileCount, filePath in enumerate(train_niiPaths[:]):
    if "No treatment" in filePath:
        labels[filePath] = 0
    elif "Ex" in filePath:
        if "/1m" in filePath:
            labels[filePath] = 1
        elif "/5m" in filePath:
            labels[filePath] = 2
        elif "/30m" in filePath:
            labels[filePath] = 3
    else:
        if "/1m" in filePath:
            labels[filePath] = 4
        elif "/5m" in filePath:
            labels[filePath] = 5
        elif "/30m" in filePath:
            labels[filePath] = 6

    
    #print("Saving Labels: " + str(fileCount+1) + ". Label:" + str(label) + "for filepath " + str(filePath))
    

    
partition['train'] = train_niiPaths

test_rootdir = "/content/drive/My Drive/Classifier_valid/"
test_niiPaths = []
fill_value = 0
num_files= len(test_niiPaths) 


for subdir, dirs, files in os.walk(test_rootdir):
    for file in files:
        if(file.endswith("nii.gz")):
                test_niiPaths.append(os.path.join(subdir, file))


for fileCount, filePath in enumerate(test_niiPaths[:]):
    if "No treatment" in filePath:
        labels[filePath] = 0
    elif "Ex" in filePath:
        if "/1m" in filePath:
            labels[filePath] = 1
        elif "/5m" in filePath:
            labels[filePath] = 2
        elif "/30m" in filePath:
            labels[filePath] = 3
    else:
        if "/1m" in filePath:
            labels[filePath] = 4
        elif "/5m" in filePath:
            labels[filePath] = 5
        elif "/30m" in filePath:
            labels[filePath] = 6

partition['validation'] = test_niiPaths

#partition['test'] = test_niiPaths    //note: we are using "test" data as validation in our case, would need to make another "D:/INS-1E Source Images/Unlabeled_validation" folder


img_width, img_height = 128, 128
train_data_dir = "/content/drive/My Drive/Classifier_train/"
validation_data_dir = "/content/drive/My Drive/Classifier_valid/"
nb_train_samples = len(train_niiPaths) 
nb_validation_samples = len(test_niiPaths) 
batch_size = 32
epochs = 10


# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

for layer in model.layers[:5]:
    layer.trainable = False
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation="softmax")(x)

model_final = Model(input = model.input, output = predictions)
model_final.summary()

model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])



Number of source filePaths: 196
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D) 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:112: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [0]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

# Train model on dataset
model_final.fit_generator(generator=training_generator,
                    validation_data=validation_generator, epochs=200, verbose=1)


#score = model_final.evaluate(x_test, y_test, verbose=1)

model.save_weights('G:/classifier_models/model_weights_04_03_2020.h5') 

print("saved")


with open('G:/classifier_models/model_architecture_04_03_2020.json', 'w') as f:

    f.write(model.to_json())

from keras.models import model_from_json


# Model reconstruction from JSON file

with open('G:/classifier_models/model_architecture_04_03_2020.json', 'r') as f:

    model = model_from_json(f.read())


# Load weights into the new model

model.load_weights('G:/classifier_models/model_weights_04_03_2020.h5') #change names


# confusion matrix

from sklearn.metrics import confusion_matrix

test_labels = np.zeros((len(test_labelArray)))

for i,label in enumerate(test_labelArray):

    for j in range(7):

        if label[j] == 1:

            test_labels[i] = j

print(ynew)

print(test_labels)

confusionArray = confusion_matrix(test_labels, ynew)

print(confusionArray)

Epoch 1/200
